In [1]:
#***********************************************#
#             Mapping by bowtie2                #
#***********************************************#



rawpath="/home/asus/Desktop/CHRF_Project/Bioinformatics_Traning/Advance_Bioinformatics_Traning/Data"



Ref_SPN='/home/asus/Desktop/CHRF_Project/Bioinformatics_Traning/Advance_Bioinformatics_Traning/Mapping/Ref_SPN'
refname=NZ_CP035252_SPN
ref=/home/asus/Desktop/CHRF_Project/Bioinformatics_Traning/Advance_Bioinformatics_Traning/Mapping/Ref_SPN/NZ_CP035252_SPN.fa


In [2]:
# Build Index (speed up the alignment)
mkdir -p -v sys$Ref_SPN\/Index/
bowtie2-build $Ref_SPN\/$ref $Ref_SPN\/Index/index
Index=$Ref_SPN/Index/

mkdir: cannot create directory ‘/home/asus/Desktop/CHRF_Project/Bioinformatics_Traning/Advance_Bioinformatics_Traning/Mapping/Ref_SPN/Index/’: File exists
Settings:
  Output files: "/home/asus/Desktop/CHRF_Project/Bioinformatics_Traning/Advance_Bioinformatics_Traning/Mapping/Ref_SPN/Index/index.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  /home/asus/Desktop/CHRF_Project/Bioinformatics_Traning/Advance_Bioinformatics_Traning/Mapping/Ref_SPN//home/asus/Desktop/CHRF_Project/Bioinformatics_Traning/Advance_Bioinformatics_Traning/Mapping/Ref_SPN/NZ_CP035252_SPN

In [3]:

for files in $rawpath\/*
do
    name=`basename $files| cut -f 1,2 -d '_'`
    echo $name
    
    mkdir -p -v $rawpath\/$name\/5_Mapping

    fastq_1=$rawpath/$name/1_RawData/$name*\_R1_001.fastq.gz
    fastq_2=$rawpath/$name/1_RawData/$name*\_R2_001.fastq.gz
    # echo $fastq_1
    # echo $fastq_2


    echo "# Mapping the reads"
    bowtie2 -x $Index\/index -1 $fastq_1 -2 $fastq_2 -S $rawpath\/$name\/5_Mapping/$name\.sam -p 12

    echo "# Convert .sam to .bam"
    samtools view -ubS --threads 12 $rawpath\/$name\/5_Mapping/$name\.sam > $rawpath\/$name\/5_Mapping/$name\.bam

    echo "# Sort the .bam"
    samtools sort --threads 12 $rawpath\/$name\/5_Mapping/$name\.bam -o $rawpath\/$name\/5_Mapping/$name\.sorted.bam
    
    echo "# Index sorted BAM alignments"
    samtools index -@ 72 $rawpath\/$name\/5_Mapping/$name\.sorted.bam
    
    
    echo "# Call SNPs and Indels"
    bcftools mpileup -d 1000 -Q 20 -Ou -Bf $ref $rawpath\/$name\/5_Mapping/$name\.sorted.bam | bcftools call -cv --threads 72 | bcftools view -v snps -g hom --threads 72 > $rawpath\/$name\/5_Mapping/$name\.raw.vcf


done

SPN_0140
# Mapping the reads
2006858 reads; of these:
  2006858 (100.00%) were paired; of these:
    476166 (23.73%) aligned concordantly 0 times
    1473472 (73.42%) aligned concordantly exactly 1 time
    57220 (2.85%) aligned concordantly >1 times
    ----
    476166 pairs aligned concordantly 0 times; of these:
      177908 (37.36%) aligned discordantly 1 time
    ----
    298258 pairs aligned 0 times concordantly or discordantly; of these:
      596516 mates make up the pairs; of these:
        519464 (87.08%) aligned 0 times
        55968 (9.38%) aligned exactly 1 time
        21084 (3.53%) aligned >1 times
87.06% overall alignment rate
# Convert .sam to .bam
# Sort the .bam
[bam_sort_core] merging from 0 files and 12 in-memory blocks...
# Index sorted BAM alignments
# Call SNPs and Indels
Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 100